<a href="https://colab.research.google.com/github/sapta1989/Google_Earth_engine/blob/master/Sentinel2_NDVI_batch_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
ee.Authenticate()
import ee
ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=wBIk0sHPnTt8qKSInstnUsI5HT71qluk-lp1I6BX-kA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3wGzAIEqvJpLo4FRbQDlKS0q8055hF9VlIbIo9CV82-kMVLNgmvhw5E

Successfully saved authorization token.


In [70]:
tiles = ["44QLG", "44QLE", "44QLD", "44QKE", "44PKC", "44QKG", "44QKD", "44QMF", "44QKF", "44QME", "44QMD"]

##print(sentinel2_tile_bound)
startDate = '2019-09-01'
endDate = '2020-05-31'
## function for cloud masking
def maskCloudAndShadows(image):
  cloudProb = image.select('MSK_CLDPRB')
  snowProb = image.select('MSK_SNWPRB')
  cloud = cloudProb.lt(5)
  snow = snowProb.lt(5)
  scl = image.select('SCL')
  shadow = scl.eq(3) ## 3 = cloud shadow
  cirrus = scl.eq(10) ## 10 = cirrus
  ## Cloud probability less than 5% or cloud shadow classification
  mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
  return image.updateMask(mask)
for tile in tiles:
  sentinel2_tile_bound = ee.FeatureCollection('users/saptarshi/Telangana_Sentinel2_boundary').filter(ee.Filter.eq('Name', tile))
  ## Define an example collection.
  collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(startDate, endDate).map(maskCloudAndShadows).filter(ee.Filter.eq('MGRS_TILE', tile))
  #collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(startDate, endDate).filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 50)).map(maskCloudAndShadows).filter(ee.Filter.eq('MGRS_TILE', tile))

  ##NDVIcollection = collection.map(calculateNDVI)
  count = int(collection.size().getInfo())
  ##print("Total number of images: {}".format(count))
  img_lst = collection.toList(count)
  ##print(img_lst)
  for i in range(0, count):
      image = ee.Image(collection.toList(count).get(i))
      region = image.geometry() #sentinel2_tile_bound.geometry() # roi
      dateString = image.get('system:index').getInfo()
      ndvi = image.normalizedDifference(['B8', 'B4'])
      ndvi2 = ndvi.multiply(10000).toInt16()

      name = "Seninel2_NDVI_T" + tile + "_" + dateString[:-34] + "-" + dateString[-34:-32] + "-" + dateString[-32:-30] # set the output image filename
      #print(name)
      # Set configration parameters for output image
      task_config = {
      ##'folder': 'Sentinel2-NDVI_data', # output Google Drive folder
      'region': region,     # roi 
      'scale': 10,       # image resolution
      'crs': 'EPSG:4326',
      'crs': 'EPSG:4326',
      'maxPixels': 1.0E13,
      'fileFormat': 'GeoTIFF'
      }
      task = ee.batch.Export.image(ndvi2.clip(region), name, task_config)
      task.start()
      print (name + " is exported")
  print("Process over for tile no: " + tile)

Seninel2_NDVI_T44QLG_2019-09-01 is exported
Seninel2_NDVI_T44QLG_2019-09-06 is exported
Seninel2_NDVI_T44QLG_2019-09-11 is exported
Seninel2_NDVI_T44QLG_2019-09-16 is exported
Seninel2_NDVI_T44QLG_2019-09-21 is exported
Seninel2_NDVI_T44QLG_2019-09-26 is exported
Seninel2_NDVI_T44QLG_2019-10-01 is exported
Seninel2_NDVI_T44QLG_2019-10-06 is exported
Seninel2_NDVI_T44QLG_2019-10-11 is exported
Seninel2_NDVI_T44QLG_2019-10-16 is exported
Seninel2_NDVI_T44QLG_2019-10-21 is exported
Seninel2_NDVI_T44QLG_2019-10-26 is exported
Seninel2_NDVI_T44QLG_2019-10-31 is exported
Seninel2_NDVI_T44QLG_2019-11-05 is exported
Seninel2_NDVI_T44QLG_2019-11-10 is exported
Seninel2_NDVI_T44QLG_2019-11-15 is exported
Seninel2_NDVI_T44QLG_2019-11-20 is exported
Seninel2_NDVI_T44QLG_2019-11-25 is exported
Seninel2_NDVI_T44QLG_2019-11-30 is exported
Seninel2_NDVI_T44QLG_2019-12-05 is exported
Seninel2_NDVI_T44QLG_2019-12-10 is exported
Seninel2_NDVI_T44QLG_2019-12-15 is exported
Seninel2_NDVI_T44QLG_2019-12-20 